# Analysis-of-2024-French-Legislatives-Elections

İmport Modules

In [39]:
import pandas as pd
import sqlite3
import google.generativeai as genai
import csv
import dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import Tool


# Gathering Data

In [40]:
csv_file = 'resultats-definitifs-par-regions.csv'
df = pd.read_csv(csv_file, sep=';', decimal=',')

# create a SQLite database and connect to it
db_file = 'elections.db'
conn = sqlite3.connect(db_file)

# write the DataFrame to a SQLite table
df.to_sql('elections', conn, if_exists='replace', index=False)
conn.close()
print(f"Data from {csv_file} has been written to {db_file} in the 'elections' table.")


Data from resultats-definitifs-par-regions.csv has been written to elections.db in the 'elections' table.


# Using the LLM to Write and Run SQL

In [41]:
dotenv.load_dotenv(override=True)


db = SQLDatabase.from_uri("sqlite:///elections.db")


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

db_chain = SQLDatabaseChain(
    llm=llm,
    database=db,
    verbose=True,
    return_direct=True
)
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description",
    max_iterations=1,
    early_stopping_method="generate",
    verbose=True
)

response = agent.run("How many voters are in France?")
print(response)



> Entering new AgentExecutor chain...


/home/codespace/.python/current/lib/python3.12/site-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


Thought: I need to query the database for the number of voters in France.

Action: SQL Database
Action Input: SELECT COUNT(*) FROM voters WHERE country = 'France';

> Entering new SQLDatabaseChain chain...
SELECT COUNT(*) FROM voters WHERE country = 'France';
SQLQuery:SQLQuery: SELECT COUNT(*) FROM elections;
SQLResult: [(18,)]
> Finished chain.

Observation: [(18,)]
Thought:Thought: I now know the final answer. The query returned a count of 18.

Final Answer: There are 18 voters in France.

> Finished chain.
There are 18 voters in France.
